In [313]:
#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [314]:
# N = 14
# data_df = pd.read_csv('../SPECT.train',header=None)
# features = df.loc[:,df.columns!=0]
# label = df[0]
# label_column = 0

In [315]:
N = 14
data_df = pd.read_csv("../haberman.data",header=None)
data_df.fillna(data_df.median(),inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
label_column = 'Survival_status'
noise_frac = 0.3

In [316]:
def add_noise(data, col):
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = noise_frac)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)
    return mislabeled,noisy_indices

In [317]:
mislabeled,noisy_indices = add_noise(data_df, label_column)

In [318]:
def distance_from_centroid(df,centroids,data_label):
    d = defaultdict(list)
    for index in df.index:
        dist = 0
        for i in range(len(centroids[0])):
            dist+=(centroids[df.loc[index,'cluster']][i] - df.loc[index,df.columns[i]])**2
        d[df.loc[index,'cluster']].append([dist,index,data_label[index]])
    return d

In [319]:
def ratio_in_cluster(features,data_label,n):
    majority_label_in_cluster = defaultdict(None)
    km = KMeans(n_clusters = n)
    km.fit(features)
    label = km.labels_
    print(len(label))
    print(features.shape)
    features['cluster'] = label
    
    distance = distance_from_centroid(features,km.cluster_centers_,data_label)
    
    for i in distance.keys():
        distance[i] = sorted(distance[i],key=lambda x: x[0])
        
    normalized_distance = dict(distance)
    label_ratio_count = {}
    
    for i in normalized_distance:
        temp = [0,0]
        for j in range(len(normalized_distance[i])):
            normalized_distance[i][j][0] = normalized_distance[i][j][0]/normalized_distance[i][-1][0]
            temp[normalized_distance[i][j][-1]] +=1

        temp[0],temp[1] = temp[0]/sum(temp),temp[1]/sum(temp)
        label_ratio_count[i] = temp
        
        if temp[0] > temp[1] and temp[0]>=0.7:
            majority_label_in_cluster[i] = 0
        elif temp[0] < temp[1] and temp[1]>=0.7:
            majority_label_in_cluster[i] = 1
        else:
            majority_label_in_cluster[i] = None

    return(normalized_distance, majority_label_in_cluster)
    #return pd.DataFrame(data=label_ratio_count),majority_label_in_cluster  
    

In [320]:
normalized_distance, majority_label_in_cluster = ratio_in_cluster(mislabeled.loc[:,mislabeled.columns!=label_column],mislabeled[label_column],N)

306
(306, 3)


In [321]:
points_removed = []
reduced_points = defaultdict(list)
for cluster_id in normalized_distance.keys():
    count = 0
    for point in normalized_distance[cluster_id]:
        if count/len(normalized_distance[cluster_id]) > 0.75:
            if point[-1] != majority_label_in_cluster[cluster_id] and majority_label_in_cluster[cluster_id]!=None:
                points_removed.append(point)
                continue
            else:
                reduced_points[cluster_id].append(point)
                count+=1
        else:
            reduced_points[cluster_id].append(point)
            count +=1

In [322]:
# result = []
# for cluster_id in normalized_distance:
#     print(len(normalized_distance[cluster_id]), len(reduced_points[cluster_id]))
#     print((len(reduced_points[cluster_id]) * 1.0) -  len(normalized_distance[cluster_id]))
#     print()

In [323]:
#reduced_points

In [324]:
#majority_label_in_cluster

In [325]:
#points_removed

In [326]:
noisy_indices = set(noisy_indices)
n_n = 0
p_n = 0
for point in points_removed:
    if point[2] in noisy_indices:
        n_n+=1
    else:
        p_n+=1
p_p = (len(mislabeled) - len(noisy_indices)) - p_n
n_p = len(noisy_indices) - n_n
print("# Observation in Culprit",len(points_removed))
print("Noise to Noise",n_n)
print("Pure to Noise",p_n)
print("Noise to Pure",n_p)
print("Pure to Pure",p_p)
print('Pure %',100 * p_p/(len(mislabeled)-len(points_removed)))
print('Noisy %',100 * n_p/(len(mislabeled)-len(points_removed)))
print('Cleaned %',100 * n_n/(len(noisy_indices)))


# Observation in Culprit 5
Noise to Noise 5
Pure to Noise 0
Noise to Pure 87
Pure to Pure 214
Pure % 71.09634551495017
Noisy % 28.903654485049834
Cleaned % 5.434782608695652


# 